In [1]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import numpy as np
import re
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
import gensim
from gensim import models
from gensim import corpora
import lda

unable to import 'smart_open.gcs', disabling that module


In [2]:
train_input = pd.read_csv('s3://recsys-challenge-2020/train_input.csv')
val_input = pd.read_csv('s3://recsys-challenge-2020/val_input.csv')
test_input = pd.read_csv('s3://recsys-challenge-2020/test_input.csv')

In [3]:
train_input.head()

,tweet_text,name
0,envivo buenas noches comienza una nueva edició...,pt
1,esta bebé nació con una marca que la ha hecho ...,pt
2,elsalvador el tiempo y el olvido juegan en con...,pt
3,menciona a la mane _ acasore de tu karime _ ac...,pt
4,ddn apocalipsis zombi tras visitar otra agenci...,pt


In [4]:
train_input.index = train_input.index.astype(str) + '_train'
val_input.index = val_input.index.astype(str) + '_val'
test_input.index = test_input.index.astype(str) + '_test'

In [5]:
all_data = pd.concat([train_input, val_input, test_input])

In [6]:
all_data.head()

,tweet_text,name
0_train,envivo buenas noches comienza una nueva edició...,pt
1_train,esta bebé nació con una marca que la ha hecho ...,pt
2_train,elsalvador el tiempo y el olvido juegan en con...,pt
3_train,menciona a la mane _ acasore de tu karime _ ac...,pt
4_train,ddn apocalipsis zombi tras visitar otra agenci...,pt


In [9]:
import fasttext

In [10]:
len(all_data)

163202921

In [11]:
>>> import fasttext.util
>>> fasttext.util.download_model('en', if_exists='ignore')  # English
>>> ft = fasttext.load_model('cc.en.300.bin')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 (1.32%) [>                                                  ]                                                  ]                                                  ]                                                  ]>                                                  ]                                                  ]                                                  ]                                                  ]                                                  ]>                                                  ]]                                                  ]>                                                  ]                                                  ]]                                                  ]>                                                  ]                                                  ]                                                  ]]                                                  ]                                                  ]             

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 (97.54%) [================================================>  ]  ]]]  ]>  ]]================================================>  ]  ]  ]  ]  ]>  ]]]]  ]  ]  ]================================================>  ]  ]================================================>  ]  ]  ]  ]  ]  ]  ]  ]]]>  ]  ]  ]  ]  ]  ]  ]]================================================>  ]  ]  ]  ]  ]  ]  ]]  ]  ]================================================>  ]]]  ]================================================>  ]]  ]  ]  ]================================================>  ]]  ]  ]  ]  ]  ]  ]  ]  ]================================================>  ]  ]  ]  ]  ]]  ]  ]]>  ]  ]  ]  ]  ]>  ]  ]  ]  ]  ]  ]================================================>  ]  ]]  ]  ]  ]  ]  ]  ]  ]]  ]  ]  ]]  ]================================================>  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]]  ]  ]  ]  ]  ]  ]]  ]  ]  ]  ]  ]================================================>  ]  ]  ]  ]  ]  ]]  ]  ]]  ]  ]  ]  ]  ]  ]  ]  ]  ]]>  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
fasttext.util.reduce_model(ft, 20)

In [20]:
ft.get_sentence_vector('I am')

array([-0.29566437, -0.42625526,  0.02700479, -0.19624013,  0.54058987,
        0.24732476, -0.21765901, -0.13427377, -0.23517832,  0.01904584,
       -0.13821861,  0.02259632,  0.0016915 , -0.08761315, -0.268183  ,
        0.10526881,  0.15189722,  0.07793003, -0.21797225, -0.02783458],
      dtype=float32)

In [21]:
full_input = all_data.sort_values(by='name')

In [22]:
languages = list(full_input.name.unique())

In [27]:
def get_fasttext_sentence_embedding(row):
    if pd.isna(row):
        return np.zeros(20)
    return ft.get_sentence_vector(row)

In [24]:
all_lang_output = pd.DataFrame()

In [25]:
lang_output = pd.DataFrame()
lang_input = full_input.loc[full_input.name == 'en']

AttributeError: 'Series' object has no attribute 'tweet_text'

In [ ]:
lang_output['sentence_embedding'] = lang_input.apply(lambda x: get_fasttext_sentence_embedding(x.tweet_text), axis = 1)

In [ ]:
all_lang_output = all_lang_output.concat([all_lang_output, lang_output])